In [26]:
import fitbit
import gather_keys_oauth2 as Oauth2
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
import warnings
import json
import requests
warnings.filterwarnings('ignore')

# Load the JSON file
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

    

# Correctly access client_id and client_secret
CLIENT_ID = config['client_id']
CLIENT_SECRET = config['client_secret']
ACCESS_TOKEN = config['access_token']

# Assuming CLIENT_ID, CLIENT_SECRET, and ACCESS_TOKEN are already set
header = {'Authorization': 'Bearer {}'.format(ACCESS_TOKEN)}


In [27]:
DATE_FORMAT = '%Y-%m-%d'
    
def prettyPrintDate(date):
    return date.strftime(DATE_FORMAT);


def safeGet(obj, key, defaultVal = np.nan):
    return obj.get(key, defaultVal)  


def filterArrObj(arrList, keyName, keyValue):
    for arrItem in arrList:
        if arrItem.get(keyName) == keyValue:
            return arrItem
    return {}

# Define the endpoints
profile_url = "https://api.fitbit.com/1/user/-/profile.json"
activities_url = "https://api.fitbit.com/1/user/-/activities/date/2023-12-17.json"
sleep_url = "https://api.fitbit.com/1.2/user/-/sleep/date/2023-12-17.json"
heart_rate_url = "https://api.fitbit.com/1/user/-/activities/heart/date/2023-12-17/1d.json"

# Function to make a GET request
def get_fitbit_data(url):
    response = requests.get(url, headers=header)
    return response.json()


def getActivities(date):
    activitiyResponse = get_fitbit_data(f"https://api.fitbit.com/1/user/-/activities/date/{date}.json")
    
    activitySummary = activitiyResponse.get('summary',{})
    activityData = {
        'Calories Burned':safeGet(activitySummary,'caloriesOut'),
        'Calories BMR': safeGet(activitySummary,'caloriesBMR'),
        'Resting Heart Rate':safeGet(activitySummary,'restingHeartRate'),
        'Steps':safeGet(activitySummary,'steps'),
        'Distance (Km)':filterArrObj(activitySummary.get('distances', []), 'activity', 'total').get('distance', np.nan),
        'Elevation (Ft)':activitySummary.get('elevation',{}),
        'Floors':activitySummary.get('floors',{}),
        'Minutes Sedentary':activitySummary.get('sedentaryMinutes',{}),
        'Minutes Lightly Active':activitySummary.get('lightlyActiveMinutes',{}),
        'Minutes Fairly Active':activitySummary.get('fairlyActiveMinutes',{}),
        'Minutes Very Active':activitySummary.get('veryActiveMinutes',{}),
        'Activity Calories': activitySummary.get('activityCalories',{}),
        'Active Score': activitySummary.get('activeScore',{}),
        'Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('minutes', np.nan),
        'Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('caloriesOut', np.nan),
        'Fat Burn minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('minutes', np.nan),
        'Fat Burn calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('caloriesOut', np.nan),
        'Peak minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('minutes', np.nan),
        'Peak calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('caloriesOut', np.nan),
        'Normal Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('minutes', np.nan),
        'Normal Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('caloriesOut', np.nan),
    }
    
    
    return activityData
def getSleep(date):
    sleepResponse = get_fitbit_data(f"https://api.fitbit.com/1/user/-/sleep/date/{date}.json")
    
    sleepData = {}
    for sleepLog in sleepResponse.get('sleep', []):
        if sleepLog.get('isMainSleep'):
            sleepLevelsSummary = sleepResponse.get('summary', {}).get('stages', {})

            sleepData['Sleep Efficiency'] = safeGet(sleepLog, 'efficiency')
            sleepData['Minutes Asleep'] = safeGet(sleepLog, 'minutesAsleep')
            sleepData['Minutes to fall asleep'] = safeGet(sleepLog, 'minutesToFallAsleep')
            sleepData['Sleep Start time'] = safeGet(sleepLog, 'startTime')
            sleepData['Sleep End time'] = safeGet(sleepLog, 'endTime')
            
            sleepData['Time in bed'] = safeGet(sleepLog, 'timeInBed')

            sleepData['Minutes Deep sleep'] = sleepLevelsSummary.get('deep', 0)
            sleepData['Deep sleep count'] = sleepLevelsSummary.get('deep', {})
    #             sleepData['% Deep sleep'] = percent(safeGet(sleepData, 'Minutes Deep sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes Light sleep'] = sleepLevelsSummary.get('light', 0)
            sleepData['Light sleep count'] = sleepLevelsSummary.get('light', {})
    #             sleepData['% Light sleep'] = percent(safeGet(sleepData, 'Minutes Light sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes REM sleep'] = sleepLevelsSummary.get('rem', 0)
            sleepData['REM sleep count'] = sleepLevelsSummary.get('rem', {})
    #             sleepData['% REM sleep'] = percent(safeGet(sleepData, 'Minutes REM sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes Asleep'] = sleepData['Minutes Deep sleep'] + sleepData['Minutes Light sleep'] + sleepData['Minutes REM sleep']
            sleepData['Minutes Awake'] = sleepLevelsSummary.get('wake', {})
            sleepData['Minutes Awake count'] = sleepLevelsSummary.get('wake', {})      
    return sleepData

def getDateData(date):
    weekDayNum = date.isoweekday()
    return {
        'Day of Week': weekDayNum,
        'Is Weekday': weekDayNum<6,
        'Is Weekend': weekDayNum>5,
        'Date': prettyPrintDate(date)
        };

def fetchAllData(date):
    dateStr = prettyPrintDate(date)
    
    print("Fetching fitbit data for: " + dateStr)
    
    nextDate = date + timedelta(days=1)
    sleepData = getSleep(prettyPrintDate(nextDate))
     
    activitiesData = getActivities(dateStr)
    dateData = getDateData(date)
    
    
    mergedData = {**sleepData, **activitiesData, **dateData} #merging the dictionaries together
    return mergedData

In [28]:
# initiating an empty dataframe to store the final results
database = pd.DataFrame()


def fetchAndAppendToDb(date, database):    
    mergedData = fetchAllData(date)
    for key, value in mergedData.items():
        if value == {}:
            mergedData[key] = np.nan
    # Convert mergedData to DataFrame and then concatenate
    print(mergedData)
    mergedData_df = pd.DataFrame([mergedData])
    database = pd.concat([database, mergedData_df], ignore_index=True)
    return database

def fetchData(database):    
    #here we are extracting the data for the last 1 week
    StartDate = (datetime.datetime.today() - datetime.timedelta(720)) #date from which you want to start extracting data
    EndDate = (datetime.datetime.today() - datetime.timedelta(620)) #date upto which you want to extract data 
    dateToFetch = StartDate + timedelta(days=1)
    while dateToFetch < EndDate:
        database = fetchAndAppendToDb(dateToFetch, database)
        dateToFetch = dateToFetch + timedelta(days=1)
    print("----------------------------------------------")
    print("Data fill completed! 👍👍")
    return database



In [29]:
database = fetchData(database)
database

Fetching fitbit data for: 2023-08-08
{'Sleep Efficiency': 95, 'Minutes Asleep': 450, 'Minutes to fall asleep': 0, 'Sleep Start time': '2023-08-08T23:48:00.000', 'Sleep End time': '2023-08-09T08:05:30.000', 'Time in bed': 497, 'Minutes Deep sleep': 106, 'Deep sleep count': 106, 'Minutes Light sleep': 218, 'Light sleep count': 218, 'Minutes REM sleep': 126, 'REM sleep count': 126, 'Minutes Awake': 47, 'Minutes Awake count': 47, 'Calories Burned': 4459, 'Calories BMR': 1795, 'Resting Heart Rate': 56, 'Steps': 14845, 'Distance (Km)': 9.24, 'Elevation (Ft)': 286.51, 'Floors': 94, 'Minutes Sedentary': 457, 'Minutes Lightly Active': 315, 'Minutes Fairly Active': 119, 'Minutes Very Active': 102, 'Activity Calories': 3182, 'Active Score': -1, 'Cardio minutes': 5, 'Cardio calories': 53.86608, 'Fat Burn minutes': 285, 'Fat Burn calories': 1929.07899, 'Peak minutes': 0, 'Peak calories': 0, 'Normal Cardio minutes': 1097, 'Normal Cardio calories': 2410.2577, 'Day of Week': 2, 'Is Weekday': True, 'Is

,Sleep Efficiency,Minutes Asleep,Minutes to fall asleep,Sleep Start time,Sleep End time,Time in bed,Minutes Deep sleep,Deep sleep count,Minutes Light sleep,Light sleep count,...,Fat Burn minutes,Fat Burn calories,Peak minutes,Peak calories,Normal Cardio minutes,Normal Cardio calories,Day of Week,Is Weekday,Is Weekend,Date
0,95.0,450.0,0.0,2023-08-08T23:48:00.000,2023-08-09T08:05:30.000,497.0,106.0,106.0,218.0,218.0,...,285.0,1929.07899,0.0,0.0000,1097.0,2410.25770,2,True,False,2023-08-08
1,93.0,352.0,0.0,2023-08-10T01:12:00.000,2023-08-10T08:25:30.000,433.0,60.0,60.0,224.0,224.0,...,187.0,1262.86032,0.0,0.0000,758.0,1383.56024,3,True,False,2023-08-09
2,99.0,291.0,0.0,2023-08-10T23:52:00.000,2023-08-11T05:36:00.000,344.0,58.0,58.0,191.0,191.0,...,164.0,897.26924,3.0,41.1477,1136.0,1996.41159,4,True,False,2023-08-10
3,75.0,406.0,0.0,2023-08-12T06:55:00.000,2023-08-12T11:41:30.000,286.0,34.0,34.0,344.0,344.0,...,109.0,679.06174,0.0,0.0000,986.0,2254.14582,5,True,False,2023-08-11
4,94.0,420.0,0.0,2023-08-13T01:00:30.000,2023-08-13T09:01:30.000,481.0,72.0,72.0,275.0,275.0,...,69.0,297.63503,0.0,0.0000,1370.0,2019.47924,6,False,True,2023-08-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4,True,False,2023-12-14
129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,True,False,2023-12-15
130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6,False,True,2023-12-16
131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7,False,True,2023-12-17


In [30]:
database.to_excel('Fitbit_data.xlsx')